In [ ]:
import psycopg2
import configparser
import os
import pandas as pd
from sqlalchemy import create_engine, text
import subprocess
import sys
import papermill as pm
import json
import math
from psycopg2.extras import execute_batch
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
def get_db_config():
    config = configparser.ConfigParser()
    config.read('../api_data/db.ini')
    
    return {
        'database': config['postgresql']['database'],
        'user': config['postgresql']['user'],
        'password': config['postgresql']['password'],
        'host': config['postgresql']['host'],
        'port': config['postgresql']['port']
    }
# connect to database 
db_config = get_db_config()
conn = psycopg2.connect(
    database=db_config['database'],
    user=db_config['user'],
    password=db_config['password'],
    host=db_config['host'],
    port=db_config['port']
)
cursor = conn.cursor()
print("Database connected successfully")

In [ ]:
# Create SQLAlchemy engine for pandas to_sql functionality
CONNECTION_STR = (
    f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}"
    f"@{db_config['host']}:{db_config['port']}/{db_config['database']}"
)
ENGINE = create_engine(CONNECTION_STR)

DATAFRAME = None

def log_reg_features():
    return

def win_probability():
    log_model =  LogisticRegression()
    lambda_parameter = None
    fitted = False
    

def preprocessing():
    global DATAFRAME
    conn = ENGINE.raw_connection()
    cursor = conn.cursor()
    query = """
        select home_fouls, away_fouls, home_3pt_percentage, away_3pt_percentage,
        home_free_throw_percentage, away_free_throw_percentage, home_team_fouls, away_team_fouls, possessions_lead_or_trail,
        home_ft_bonus, away_ft_bonus, time_left from play_by_play_q4
        where period = 4
    """
    db = pd.read_sql(query, ENGINE)
    array_type_columns = ["home_3pt_percentage", "away_3pt_percentage", "home_free_throw_percentage", "away_free_throw_percentage","home_fouls", "away_fouls"]
    expanded_dfs = []

    for col in array_type_columns:

        arrays = db[col].apply(lambda x: x if isinstance(x, list) else [])
        
        max_len = arrays.map(len).max()

        padded = arrays.apply(lambda x: x + [np.nan] * (max_len - len(x)))

        expanded = pd.DataFrame(
            padded.tolist(),
            columns=[f"{col}_{i+1}" for i in range(max_len)]
        )
        expanded_dfs.append(expanded)
    db = pd.concat([db.drop(columns=array_type_columns)] + expanded_dfs, axis=1)
    DATAFRAME = db
    print(db.head())

def pca():
    X = DATAFRAME
    X = X.dropna()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=.90)
    X_pca = pca.fit_transform(X_scaled)
    print("Original shape:", X.shape)
    print("Reduced shape:", X_pca.shape)
    print("Explained variance ratio:", pca.explained_variance_ratio_)

    

preprocessing()
pca()



